# Benchmarking an open-source ECG classifier

This report benchmarks the difference in performance of an open-source ECG 
classification model on general population vs athletic population cohorts.

The model selected was developed by researchers at Seoul National University 
for the 2020 PhysioNet Challenge. The paper describing their approach, 
"Bag of Tricks for Electrocardiogram Classification With Deep Neural Networks", 
is a good read for novice machine learning practitioners.

## Notebook setup

In [1]:
#| code-fold: true
#| code-summary: "Click to see packages imported"
import os
import configparser
import shutil
from pathlib import Path
from zipfile import ZipFile

import wfdb
from torchinfo import summary
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

/Users/shaun/source/Thesis/PhysioNetChallenge2020/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#|include: false
# If the current working directory is the nbs/ folder, change to the project 
# root directory instead.

if Path.cwd().stem == "nbs":
    os.chdir(Path.cwd().parent)
print(f"The current working directory is {Path.cwd()}")

The current working directory is /Users/shaun/source/Thesis/PhysioNetChallenge2020


In [3]:
#| code-fold: true
#| code-summary: "Click to see local packages imported"
from src.run_12ECG_classifier import load_12ECG_model, run_12ECG_classifier
from src.data.util import get_all_records, get_predicted_findings, diagnosis_codes, codes_to_label_vector
import src.data.norwegian as norwegian

In [4]:
#|include: false
# Import configuration settings, like location of data directory.
config = configparser.ConfigParser()
if not Path("config.ini").exists():
    print("WARNING: Please generate a config.ini file by running scripts/get_datasets.py")
else:
    config.read("config.ini")
    data_dir = Path((config["datasets"]["path"])).expanduser()
    print(f"Datasets are located at {data_dir.resolve()}")

Datasets are located at /Users/shaun/source/Thesis/PhysioNetChallenge2020/data


In [5]:
#|include: false
# Extract source code and weights from zip file
# We only need the weights
model_zip_dir = data_dir / "challenge-2020" / "1.0.2" / "sources"
with ZipFile(model_zip_dir / "DSAIL_SNU.zip", 'r') as zip:
    zip.extractall(model_zip_dir)

In [6]:
#|include: false
team_dir = model_zip_dir / "DSAIL_SNU" / "PhysioNetChallenge2020_DSAIL_SNU7"

"""
To make it easier to run and modify the model, we'll copy the source code and 
model weights to the root of our git repo.

First, we'll copy the following directories from `model_dir` to the root of our git repo:
- `config` (includes confusion matrix weights, settings for data preprocessing etc.)
- `output_training_directory` (checkpoints for model weights)
"""

result = shutil.copytree(team_dir / "output_training_directory", Path.cwd() / "checkpoints" / "original", dirs_exist_ok=True)
print(result)

# Don't want to overwrite if we make changes to config later on.
try:
    shutil.copytree(team_dir / "config", Path.cwd() / "config", dirs_exist_ok=False)
except FileExistsError:
    print("config directory already exists")

/Users/shaun/source/Thesis/PhysioNetChallenge2020/checkpoints/original
config directory already exists


## Run benchmark

In [7]:
weights_dir = Path.cwd() / "checkpoints" / "original"
config_dir = Path.cwd() / "config"
# input_dir = Path.cwd() / "data" / "challenge-2020" / "1.0.2" / "training" / "georgia" / "g1"

benchmark_dir = Path.cwd() / "data" / "benchmark"
if not benchmark_dir.exists():
    benchmark_dir.mkdir()

In [8]:
# Benchmark: norwegian-athlete-ecg
input_dir = Path.cwd() / "data" / "norwegian-athlete-ecg" / "1.0.0"
output_dir = benchmark_dir / "norwegian-athlete-ecg"

if output_dir.exists():
    print(f"{output_dir} already exists. Benchmark has already been run.")
else:
    # Run benchmark using modified PhysioNet Challenge 2020 driver.py
    !python PhysioNet2020_driver.py {weights_dir} {config_dir} {input_dir} {output_dir}

/Users/shaun/source/Thesis/PhysioNetChallenge2020/data/benchmark/norwegian-athlete-ecg already exists. Benchmark has already been run.


In [9]:
# Benchmark: georgia, subset g1
input_dir = Path.cwd() / "data" / "challenge-2020" / "1.0.2" / "training" / "georgia" / "g1"
output_dir = benchmark_dir / "georgia" / "g1"

if output_dir.exists():
    print(f"{output_dir} already exists. Benchmark has already been run.")
else:
    # Run benchmark using modified PhysioNet Challenge 2020 driver.py
    output_dir.mkdir(parents=True)
    !python PhysioNet2020_driver.py {weights_dir} {config_dir} {input_dir} {output_dir}

/Users/shaun/source/Thesis/PhysioNetChallenge2020/data/benchmark/georgia/g1 already exists. Benchmark has already been run.


In [10]:
# Benchmark: CPSC 2018 (not used for training by DSAIL_SNU)
input_dir = Path.cwd() / "data" / "challenge-2020" / "1.0.2" / "training" / "cpsc_2018" / "g1"
output_dir = benchmark_dir / "cpsc_2018" / "g1"

if output_dir.exists():
    print(f"{output_dir} already exists. Benchmark has already been run.")
else:
    # Run benchmark using modified PhysioNet Challenge 2020 driver.py
    output_dir.mkdir(parents=True)
    !python PhysioNet2020_driver.py {weights_dir} {config_dir} {input_dir} {output_dir}

/Users/shaun/source/Thesis/PhysioNetChallenge2020/data/benchmark/cpsc_2018/g1 already exists. Benchmark has already been run.


## Scoring

We only care about a subset of diagnosis labels which are easily misdiagnosed in athletes.

In [11]:
# sinus_labels = [426177001, 426783006, 427084000, 427393009]
sinus_labels = [426177001, 426783006]       # Bradycardia or Normal
rbbb_labels = [713427006, 713426002]        # Incomplete RBBB, Complete RBBB
# won't do t-wave inversion, because no output for lead number provided.
athlete_labels = sinus_labels + rbbb_labels

In [12]:
# Scoring (PhysioNet Challenge datasets)
# input_dir = Path.cwd() / "data" / "challenge-2020" / "1.0.2" / "training" / "georgia" / "g1"
# output_dir = benchmark_dir / "georgia" / "g1"
input_dir = Path.cwd() / "data" / "challenge-2020" / "1.0.2" / "training" / "cpsc_2018" / "g1"
output_dir = benchmark_dir / "cpsc_2018" / "g1"

total_confusion = np.zeros((2,2), dtype=int)
for entry in get_all_records(input_dir):
    # Actual labels from PhysioNet Challenge dataset
    record = wfdb.rdrecord(input_dir / entry)
    comment_dx = record.comments[2].split(': ')[1]
    actual_findings = list(map(int, comment_dx.split(',')))
    actual_labels = codes_to_label_vector(actual_findings, athlete_labels)

    # Predicted label from model
    file = output_dir / (entry+'.csv')
    predicted_findings = get_predicted_findings(file)
    predicted_labels = codes_to_label_vector(predicted_findings, athlete_labels)

    # Hack: If no sinus rhythm findings, assume normal sinus rhythm (426783006)
    if sum(actual_labels) == 0:
        actual_labels[1] = 1
    if sum(predicted_labels) == 0:
        predicted_labels[1] = 1
    
    # Calculate confusion matrix for entry, add to total
    total_confusion += confusion_matrix(actual_labels, predicted_labels)

In [13]:
def print_classifier_metrics(tn, fp, fn, tp):
    P = tp + fn
    N = fp + tn
    print("Population")
    print("----------")
    print(f"Total population: {P+N}")
    print(f"Positive: {P}")
    print(f"Negative: {N}\n")

    acc = (tp + tn) / (P+N)
    ppv = tp / (tp + fp)
    f1 = 2 * tp / (2*tp + fp + fn)
    print("Performance")
    print("-----------")
    print(f"Accuracy: {acc}")
    print(f"Precision: {ppv}")
    print(f"F1-Score: {f1}")

In [14]:
total_confusion

array([[2699,  298],
       [ 289,  710]])

In [15]:
tn, fp, fn, tp = total_confusion.ravel()
print_classifier_metrics(tn, fp, fn, tp)

Population
----------
Total population: 3996
Positive: 999
Negative: 2997

Performance
-----------
Accuracy: 0.8531031031031031
Precision: 0.7043650793650794
F1-Score: 0.7075236671649228


In [16]:
# Scoring (norwegian-athlete-ecg)
input_dir = Path.cwd() / "data" / "norwegian-athlete-ecg" / "1.0.0"
output_dir = benchmark_dir / "norwegian-athlete-ecg"

total_confusion = np.zeros((2,2), dtype=int)
for entry in get_all_records(input_dir):
    # Actual labels from dataset
    record = wfdb.rdrecord(input_dir / entry)
    comments_c = record.comments[1]
    findings_c = norwegian.extract_findings(comments_c)

    actual_findings = norwegian.classify_relevant_findings(findings_c)
    actual_labels = codes_to_label_vector(actual_findings, athlete_labels)

    # Predicted label from model
    file = output_dir / (entry+'.csv')
    predicted_findings = get_predicted_findings(file)
    predicted_labels = codes_to_label_vector(predicted_findings, athlete_labels)

    # Hack: If no sinus rhythm findings, assume normal sinus rhythm (426783006)
    if sum(actual_labels) == 0:
        actual_labels[1] = 1
    if sum(predicted_labels) == 0:
        predicted_labels[1] = 1
    
    # Calculate confusion matrix for entry, add to total
    total_confusion += confusion_matrix(actual_labels, predicted_labels)


In [17]:
total_confusion

array([[63, 20],
       [16, 13]])

In [18]:
tn, fp, fn, tp = total_confusion.ravel()
print_classifier_metrics(tn, fp, fn, tp)

Population
----------
Total population: 112
Positive: 29
Negative: 83

Performance
-----------
Accuracy: 0.6785714285714286
Precision: 0.3939393939393939
F1-Score: 0.41935483870967744


In [19]:
predicted_findings

[713427006, 713426002]

In [20]:
for finding in predicted_findings:
    print(diagnosis_codes[finding])

Complete right bundle branch block
Incomplete right bundle branch block


In [21]:
# TODO: Collate diagnosis results into a table

## Reverse engineering the model

![](../media/DSAIL_model-architecture.jpg)

***Figure: ResNet architecture***

Ensemble of 10 neural networks...

In [22]:
model_DSAIL = load_12ECG_model(weights_dir, config_dir)

In [23]:
for thing in model_DSAIL:
    print(type(thing))

<class 'dsail.config.DataConfig'>
<class 'dsail.config.PreprocessConfig'>
<class 'dsail.config.RunConfig'>
<class 'list'>
<class 'numpy.ndarray'>


In [24]:
for net in model_DSAIL[3]:
    print(type(net))

<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>
<class 'dsail.model.model.ECG_model'>


In [25]:
for state in net.state_dict():
    print(state)

conv1.1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.0.conv1.1.weight
layer1.0.conv2.1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.num_batches_tracked
layer1.1.conv1.1.weight
layer1.1.conv2.1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.bn1.running_mean
layer2.0.bn1.running_var
layer2.0.bn1.num_batches_tracked
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.bn2.running_mean
layer2.0.bn2.running_var
layer2.0.bn2.num_batches_tracked
layer2.0.conv1.1.weight
layer2.0.conv2.1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.bn1.running_mean
layer2.1.bn1.running_var
layer2.1.bn1.num_bat

In [26]:
for module in net.modules():
    print(module)
    print("***")

ECG_model(
  (conv1): Sequential(
    (0): ConstantPad1d(padding=(5, 5), value=0)
    (1): Conv1d(12, 32, kernel_size=(11,), stride=(1,), bias=False)
  )
  (layer1): Sequential(
    (0): ResNet_Basic_Block(
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv1): Sequential(
        (0): ConstantPad1d(padding=(5, 5), value=0)
        (1): Conv1d(32, 32, kernel_size=(11,), stride=(1,), bias=False)
      )
      (conv2): Sequential(
        (0): ConstantPad1d(padding=(5, 5), value=0)
        (1): Conv1d(32, 32, kernel_size=(11,), stride=(1,), bias=False)
      )
      (shortcut): Sequential()
    )
    (1): ResNet_Basic_Block(
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=T

In [27]:
summary(net)

Layer (type:depth-idx)                   Param #
ECG_model                                --
├─Sequential: 1-1                        --
│    └─ConstantPad1d: 2-1                --
│    └─Conv1d: 2-2                       4,224
├─Sequential: 1-2                        --
│    └─ResNet_Basic_Block: 2-3           --
│    │    └─BatchNorm1d: 3-1             64
│    │    └─BatchNorm1d: 3-2             64
│    │    └─ReLU: 3-3                    --
│    │    └─Dropout: 3-4                 --
│    │    └─Sequential: 3-5              11,264
│    │    └─Sequential: 3-6              11,264
│    │    └─Sequential: 3-7              --
│    └─ResNet_Basic_Block: 2-4           --
│    │    └─BatchNorm1d: 3-8             64
│    │    └─BatchNorm1d: 3-9             64
│    │    └─ReLU: 3-10                   --
│    │    └─Dropout: 3-11                --
│    │    └─Sequential: 3-12             11,264
│    │    └─Sequential: 3-13             11,264
│    │    └─Sequential: 3-14             --
├─Sequen

## Modifying model weights